In [1]:
### Power Budget ###

## SORGENTI PRIMARIE ##

#GENERATORI
#PANNELLI SOLARI 
#SERVE LA POTENZA DI QUESTI


##SORGENTI SECONDARIE ##

#BATTERIE
#SERVE PERIODO DI ECLISSI (CIOE' QUANTA CARICA DEVE ACCUMULARE)
#PERIODO DI RICARICA 

### Thermal Sub-Systems


In [2]:
import numpy as np

### POTENZE SOTTOSISTEMI ###

potenzaCamera = 5 # (W)
potenzaDissipazioneCamera = 2 #(W)
potenzaTXTelecom = 35 # (dBm) =-= 3.162 W

#################################

##########
stefanBoltz = 5.67*10**(-8) # (W / m^2 * K^4)
solarConstant = 1370 #W/m^2
earthIRconstant = 235 #W/m^2
periodoTotale = 5770 #secondi
max_periodoOmbra = 2118.184 #periodo massimo di ombra (secondi)
min_periodoLuce = periodoTotale - max_periodoOmbra #periodo minimo di luce (secondi)
efficiency = 0.29#efficienza dei pannelli solari
degradationPerYear = 0.1 #tasso di degradazione dei pannelli solari all'anno
missionLenght = 2 # (years)
totalRequiredPower = 15# potenza totale richiesta dallo S\C
totalRequiredDayTimePower = 14# potenza totale richiesta dallo S\C di giorno
totalRequiredNightTimePower = 20# potenza totale richiesta dallo S\C di notte
numOrbiteTotali = (missionLenght*3.154*10**7)/periodoTotale #totale numero di orbite
Earth_Radius = 6370 # (km)
Orbital_Elevation = 574 #(km)

### Thermal ###
assorbitivita = 0.2 # solare, quanto calore ritiene lo S\C
emissivita = 0.2# quanto calore emette lo S\C
minWorkingTemp = -25 # (°C)
maxWorkingTemp = 70 # (°C)
albedo = 0.3 # tra 0.3 e 0.35
thermalRHO = np.arcsin(Earth_Radius/(Earth_Radius+Orbital_Elevation))
albedoConstant = 0.664 + 0.521*thermalRHO - 0.232*thermalRHO**2
F_vis = Earth_Radius/(Earth_Radius+Orbital_Elevation) * albedoConstant #fattore visibilità
AREA_TOTALE = 0.18 # (m^2)
AreaEsposta_albedo = 0.1*0.4 # (m^2) 10cm x 40cm 4U satellite
AreaEsposta_solare = 0.1*0.4 # (m^2)
G_A = albedo * F_vis * solarConstant





In [3]:
class panel:
    def __init__(self, eff, degradation_perYear, missionLenght, total_requiredPower, total_requiredDayTimePower, total_requiredNightTimePower):
        self.eff = eff
        self.degradation_perYear = degradation_perYear
        self.missionLenght = missionLenght
        self.total_requiredPower = total_requiredPower
        self.total_requiredDayTimePower = total_requiredDayTimePower
        self.total_requiredNightTimePower = total_requiredNightTimePower
        self.P_bol = pot_pannelli(eff)
        self.P_eol = panel_pow_EOL(self.P_bol, degradation_perYear, self.missionLenght)
        if self.total_requiredDayTimePower < self.total_requiredNightTimePower:
            worst = self.total_requiredNightTimePower
        else:
            worst = self.total_requiredDayTimePower
        self.panelArea = required_area(worst, self.P_eol)

class battery:
    def __init__(self, eff, DOD, missionLenght, numeroCicli):
        self.eff = eff
        self.DOD = DOD
        self.missionLenght = missionLenght
        self.numCicli = numeroCicli
        self.reqPow = panel.total_requiredNightTimePower / self.eff
        self.capacity = self.reqPow*max_periodoOmbra/3600
        self.totalCapacity = self.capacity / self.DOD


def POTENZA_TOTALE(pot_d, pot_e, t_d, t_e, eff_d, eff_e):
    return ((pot_d*t_d/eff_d)+(pot_e*t_e/eff_e))/t_d

def pot_pannelli(eff):
    return solarConstant*eff

def panel_pow_EOL(P_BOL, degrad_per_year, mission_lenght):
    degrad_per_year = (1-degrad_per_year)**mission_lenght
    return P_BOL * degrad_per_year

def batt_pow_EOL(P_BOL, degrad_per_year, mission_lenght):
    x = (1-degrad_per_year)**mission_lenght
    return P_BOL * x

def required_area(P_tot, P_EOL):
    return P_tot/P_EOL

def createPanel(eff, degradation_perYear, missionLenght, Total_requiredPower):
    P_bol = pot_pannelli(eff)*0.77
    P_eol = panel_pow_EOL(P_bol, degradation_perYear, missionLenght)
    print(P_eol)
    return required_area(Total_requiredPower, P_eol)

def caloreAssorbito_solare(alfa, A_s):
    return alfa * A_s * solarConstant

def caloreAssorbito_solareRiflesso(alfa, A_a, G_a):
    return alfa * A_a * G_a

def caloreAssorbito_IRterrestre(alfa, A_a, vis_fact,):
    return alfa * A_a * vis_fact * earthIRconstant

def caloreEmesso(emissività, Area_tot, T):
    return emissivita*Area_tot*stefanBoltz*T**4

def tempEcl(Q_i):
    Q_IR = caloreAssorbito_IRterrestre(assorbitivita, AreaEsposta_albedo, F_vis)
    Q = Q_i + Q_IR
    return (Q/AREA_TOTALE/emissivita/stefanBoltz)**0.25

def tempSol(Q_i=5.35):
    Q_IR = caloreAssorbito_IRterrestre(assorbitivita, AreaEsposta_albedo, F_vis)
    Q_S = 0 #caloreAssorbito_solare(assorbitivita, AreaEsposta_solare)
    Q_A = 0 #caloreAssorbito_solareRiflesso(assorbitivita, AreaEsposta_albedo, G_A)
    Q = Q_i + Q_IR + Q_A + Q_S
    return (Q/AREA_TOTALE/emissivita/stefanBoltz)**0.25

In [4]:
a = panel(efficiency, degradationPerYear, missionLenght, totalRequiredPower, totalRequiredDayTimePower, totalRequiredNightTimePower)
print(f"{a.panelArea} m^2")

T_SOLE = tempSol()

print(f"T_SOLE = {T_SOLE} K")

Q_IR = caloreAssorbito_IRterrestre(assorbitivita, AreaEsposta_albedo, F_vis)
Q_S = caloreAssorbito_solare(assorbitivita, AreaEsposta_solare)
Q_A = caloreAssorbito_solareRiflesso(assorbitivita, AreaEsposta_albedo, G_A)

print(f"EOL: {batt_pow_EOL(pot_pannelli(0.295)*0.77, 0.1, 2)}    Area Pannelli: {createPanel(0.295, 0.1, 2, 4.39)}")

0.06214789334178545 m^2
T_SOLE = 241.98440936888392 K
252.068355
EOL: 252.068355    Area Pannelli: 0.01741591085481555
